In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Matrices como transformaciónes lineales

Si tenemos una base de datos con atributos continuos podemos representar cada ejemplo como un vector $\{v_i\}$, $i=1,2,\ldots,M$.

Asumamos que $v_i \in \mathbb{R}^2$. 

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 4), tight_layout=True)
ax.set_aspect('equal')
ax.set_xlim([-3, 3]); ax.set_ylim([-3, 3]); 
v = np.random.randn(10, 2)
for v_ in v:
    ax.arrow(0, 0, v_[0], v_[1], color='k', **arrow_args)
ax.scatter(v[:, 0], v[:, 1]);

Podemos transformar nuestros datos usando una matriz cuadrada $A \in \mathbb{R}^{2\times 2}$, en este caso digamos $A = \begin{pmatrix} a_1 & a_{2} \\ a_{3} & a_4 \end{pmatrix}$

¿Qué ocurre con un vector $v = \rho \begin{pmatrix}\sin(\theta) \\ \cos(\theta)\end{pmatrix}$ cuando modificamos los componentes de la matriz?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 4), tight_layout=True)
world_axis = np.array([[1, 0], [0, 1]])
arrow_args = {'width': 0.05, 'length_includes_head': True}

def update_plot(rho, theta, a1, a2, a3, a4):
    A = [[a1, a2], [a3, a4]]
    new_world_axis = np.dot(A, world_axis)
    print(new_world_axis)
    for ax_ in ax:
        ax_.cla(); ax_.set_aspect('equal')
        ax_.set_xlim([-3, 3]); ax_.set_ylim([-3, 3]); 
    ax[0].set_title(r'$\vec v$')
    ax[1].set_title(r'$A \vec v$')
    for j in range(2):
        ax[0].arrow(0, 0, world_axis[0, j], world_axis[1, j], color='gray', alpha=0.5, **arrow_args)
        ax[1].arrow(0, 0, new_world_axis[0, j], new_world_axis[1, j], color='gray', alpha=0.5, **arrow_args)
    v = rho*np.array([np.sin(theta), np.cos(theta)])
    ax[0].arrow(0, 0, v[0], v[1], color='k', **arrow_args)
    ax[1].arrow(0, 0, v[0], v[1], color='b', **arrow_args)
    v = np.dot(A, v)
    ax[1].arrow(0, 0, v[0], v[1], color='k', **arrow_args)
    evals, evecs = scipy.linalg.eig(A)

widgets.interact(update_plot, 
                 rho=FloatSlider_nice(min=0.1, max=2, value=1.),
                 theta=FloatSlider_nice(min=-np.pi, max=np.pi, value=np.pi/4, step=1e-2),
                 a1=FloatSlider_nice(min=-2, max=2, value=1.), 
                 a2=FloatSlider_nice(min=-2, max=2, value=0.), 
                 a3=FloatSlider_nice(min=-2, max=2, value=0),
                 a4=FloatSlider_nice(min=-2, max=2, value=1));

Consideremos una transformación en particular  $A = \begin{pmatrix} 1 & 0.5 \\ 0.5 & 1 \end{pmatrix}$ 

Podemos notar que para algunos ángulos el vector transformado tiene la misma orientación que el original

Pruebe por ejemplo $\theta = \pm \pi/4 \approx \pm 0.7854$, es decir $v = \frac{\rho}{\sqrt{2}} \begin{pmatrix}  \pm 1 \\ 1\end{pmatrix}$

> Los vectores que tienen ese ángulo solo son afectados por $A$ en su magnitud 

Esos vectores se conocen como los **vectores propios** de $A$

# Problema de los valores/vectores propios

Sea una matriz cuadrada $A \in \mathbb{R}^{M\times M}$

El siguiente sistema de ecuaciones de $M$ ecuaciones

$$
\begin{split}
A \vec v &= \lambda I \vec v \\
(A - \lambda I) \vec v &= 0,
\end{split}
$$

tiene como resultado $\lambda$, los valores propios de $A$ y $\vec v$ los vectores propios de $A$

La solución no trivial de este problema se obtiene si $(A - \lambda I)$ es singular, luego su determinante

$$
|A - \lambda I | = 0
$$

que resulta en un polinomio de grado $M$ cuyas raices son $\{\lambda_i\}$, $i=1,2,\ldots, M$

Una vez determinado $\lambda_i$ se pueden usar para despejar $\vec v_i$

### Ejemplo

Para la matriz $A$ del ejemplo, si igualamos su determinante a cero tenemos

$$
(1 - \lambda)^2 - 1/4 = 3/4 - 2\lambda + \lambda^2 = 0
$$

osea $\lambda_1 = 1.5$ y $\lambda_2 = 0.5$. Luego para el primer vector propio tenemos un sistema de ecuaciones

$$
\begin{split}
-0.5v_{11} +0.5v_{12} &= 0 \\
0.5 v_{11} -0.5v_{12} &= 0
\end{split}
$$

osea $v_{11} = v_{12}$ con esto podemos construir un vector normalizado genérico $v_1 = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 \\ 1 \end{pmatrix}$

De forma equivalente para $v_2 = \frac{1}{\sqrt{2}}\begin{pmatrix} -1 \\ 1 \end{pmatrix}$

### Con computador

Podemos usar las funciones de [`scipy.linalg`](https://docs.scipy.org/doc/scipy/reference/linalg.html#eigenvalue-problems) `eig()` o `eigvals()` (y sus variantes) para resolver el sistema de ecuaciones

In [ ]:
import scipy.linalg

A = np.array([[1., 0.5], [0.5, 1]])
evals, evecs = scipy.linalg.eig(A)
display(evals, evecs)

# Descomposicion en valores propios

Existen [múltiples sistemas en física](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors#Applications) que [ocurren naturalmente](https://hubpages.com/education/What-the-Heck-are-Eigenvalues-and-Eigenvectors) como un problema de valores/vectores propios

Sin embargo la aplicación más amplia para este elemento matemático es la **descomposición en vectores propios**

> Descomponer: Expresar un elemento como una suma de partes de más simples

La descomposición que veremos a continuación usa los **vectores propios** como "las partes simples"

- ¿Cómo encontrar los vectores propios de una base de datos? ¿Qué intepretación tienen?
- ¿Cómo descomponemos nuestros datos en función de los vectores propios?
- ¿Qué ventaja tiene esta descomposición?

### Principal Component Analysis (PCA)

Es un procedimiento estadístico que busca una **transformación ortogonal** para los datos que logre **maximizar su varianza**

Para un conjunto de datos $\{x_i\}$ con $i=1,2,\ldots, M$ y $x_i \in \mathbb{R}^D$

Podemos escribirlo como una matriz $X \in \mathbb{R}^{D\times M}$

Podemos calcular su matriz de correlación como 
$$
C = \frac{1}{M} (X - \bar X) ( X - \bar X)^T
$$

donde $C \in \mathbb{R}^{D\times D}$ y $\bar X$ es la media del conjunto

Llamemos $U$ a la matriz de proyección. El problema de PCA se puede escribir como

$$
\max_U U C U^T \text{ sujeto a } U^T U = I
$$

Si usamos multiplicadores de Lagrange para incluir la restricción y luego derivamos e igualamos a cero

$$
\frac{d}{dU} U^T C U + \Lambda(I- U^T U) = CU - \Lambda U = 0
$$

donde $\Lambda = \lambda I$ y $\lambda = (\lambda_1, \lambda_2, \ldots, \lambda_D)$

> La transformación de PCA son los vectores propios de $C$

### Ejemplo: Breast cancer

Mostrar ejes proyectados

Mostrar atributos del vp 1 y 2

### Ejemplo: *Eigen-faces* o Rostros principales

In [ ]:
import sklearn.datasets
lfw_people = sklearn.datasets.fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw_people['data'].T
display(X.shape)
fig, ax = plt.subplots(3, 7, figsize=(7, 4))
for i, ax_ in enumerate(ax.ravel()):
    ax_.axis('off')
    ax_.imshow(X[:, i].reshape(50, 37), cmap=plt.cm.Greys_r);

In [ ]:
fig, ax = plt.subplots(1, figsize=(7, 2))
ax.axis('off')
ax.imshow(np.mean(X, axis=1).reshape(50, 37), cmap=plt.cm.Greys_r);

In [ ]:
X_mean = np.mean(X, axis=1, keepdims=True)
X_center = X - X_mean
C = np.dot(X_center, X_center.T)
display(C.shape)
L, U = scipy.linalg.eigh(C)

In [ ]:
idx = np.argsort(L)[::-1] # Ordenar de L más grande a más pequeño
L = L[idx]
U = U[:, idx]

In [ ]:
fig, ax = plt.subplots(3, 7, figsize=(7, 4.5), tight_layout=True)
for i, ax_ in enumerate(ax.ravel()):
    ax_.axis('off')
    ax_.set_title("{0:0.1e}".format(L[i]))
    ax_.imshow(U[:, i].reshape(50, 37), cmap=plt.cm.Greys_r);

In [ ]:
# Esto calcula los coeficientes de la imagen 0
P = np.dot(U.T, X_center[:, 0][:, None])
# Esto regenera la imagen cero a partir de sus coeficientes
Xhat = X_mean + np.dot(U, P)
display(np.allclose(X[:, 0], Xhat[:, 0], rtol=1e-2))
# ¿Cuantos coeficientes se necesitan para que X se parezca a Xhat?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(5, 2))
ax[0].axis('off'); ax[1].axis('off')
ax[0].imshow(X[:, 0].reshape(50, 37), cmap=plt.cm.Greys_r)

def update_plot(k):
    Xhat = X_mean + np.dot(U[:, :k], P[:k])
    ax[1].imshow(Xhat.reshape(50, 37), cmap=plt.cm.Greys_r)
widgets.interact(update_plot, k=SelSlider_nice(options=[1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 1850]));

## Singular Value Decomposition (SVD)

TODO

https://stats.stackexchange.com/questions/134282/relationship-between-svd-and-pca-how-to-use-svd-to-perform-pca